# AI-Tensorflow: Part 2- Real World Images Classification

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Import Data

In [26]:
# Unzip training set
zip_ref = zipfile.ZipFile('horse-or-human.zip', 'r')
zip_ref.extractall('horse-or-human')

# Unzip validation set
zip_ref = zipfile.ZipFile('validation-horse-or-human.zip', 'r')
zip_ref.extractall('validation-horse-or-human')

zip_ref.close()

In [27]:
# Define directory
import os
# Directory with training data
train_horse_dir = os.path.join('horse-or-human/horses')
train_human_dir = os.path.join('horse-or-human/humans')
# Directory with validation data
validation_horse_dir = os.path.join('validation-horse-or-human/horses')
validation_human_dir = os.path.join('validation-horse-or-human/humans')

In [30]:
# Check images name
train_horse_names = os.listdir(train_horse_dir)
print('TRAIN SET HORSES: ',train_horse_names[:5])
train_human_names = os.listdir(train_human_dir)
print('TRAIN SET HUMANS: ',train_human_names[:5])

validation_horse_names = os.listdir(validation_horse_dir)
print('VAL SET HORSES: ',validation_horse_names[:5])

validation_human_names = os.listdir(validation_human_dir)
print('VAL SET HUMANS: ',validation_human_names[:5])

TRAIN SET HORSES:  ['horse01-0.png', 'horse01-1.png', 'horse01-2.png', 'horse01-3.png', 'horse01-4.png']
TRAIN SET HUMANS:  ['human01-00.png', 'human01-01.png', 'human01-02.png', 'human01-03.png', 'human01-04.png']
VAL SET HORSES:  ['horse1-000.png', 'horse1-105.png', 'horse1-122.png', 'horse1-127.png', 'horse1-170.png']
VAL SET HUMANS:  ['valhuman01-00.png', 'valhuman01-01.png', 'valhuman01-02.png', 'valhuman01-03.png', 'valhuman01-04.png']


In [31]:
# Check the number of data
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))

print('total validation horse images:', len(os.listdir(validation_horse_dir)))
print('total validation human images:', len(os.listdir(validation_human_dir)))

total training horse images: 500
total training human images: 527
total validation horse images: 128
total validation human images: 128


## Create Model

In [44]:
# Uncomment fourth and fifth layers for input_shape = (300,300,3) which is more complex model to extract more features

model = tf.keras.models.Sequential([
    # this is the first convolution layer
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # the second convolution layer
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # the third convolution layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # the fourth convolution layer
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    
    # the fifth convolution layer
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    
    # flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # only one output neuron
    tf.keras.layers.Dense(1, activation='sigmoid')    # using sigmoid for binary classification
])

C:\Users\sajad\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
# Model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 148, 148, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     9,470,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,494,561 (36.22 MB)

 Trainable params: 9,494,561 (36.22 MB)

 Non-trainable params: 0 (0.00 B)

In [46]:
# Using RMSprop optimizer which allows to tweak learning rate parameter
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

## Data Preprocessing

In [47]:
# Be aware of change target_size to (300,300) for input_shape=(300,300,3) model
# Rescale all images by 1/255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images
train_generator = train_datagen.flow_from_directory('horse-or-human',
                               target_size=(150,150),   # all images will be resized to 300x300
                               batch_size=128,
                               class_mode='binary')
# Flow validation images
validation_generator = validation_datagen.flow_from_directory('validation-horse-or-human',
                               target_size=(150,150),   # all images will be resized to 300x300
                               batch_size=32,
                               class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [48]:
history = model.fit(train_generator,
                    steps_per_epoch=8,    # to cover all batches of data
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator,
                    validation_steps=8)

C:\Users\sajad\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5693 - loss: 4.4141 - val_accuracy: 0.5000 - val_loss: 0.7642
Epoch 2/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5234 - loss: 0.7040  
Epoch 3/15


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.7766 - loss: 0.5981 - val_accuracy: 0.5156 - val_loss: 0.8265
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8047 - loss: 0.4282
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7576 - loss: 0.4542 - val_accuracy: 0.8008 - val_loss: 0.6338
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8594 - loss: 0.2760  
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8994 - loss: 0.2924 - val_accuracy: 0.6953 - val_loss: 1.4753
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9531 - loss: 0.1333  
Epoch 9/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9579 - loss: 0.1161 - val_accuracy: 0.7891 - val_loss: 1.1882
Epoch 10/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 0.0415  
Epoch 11/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.9797 - loss: 0.0650 - val_accuracy: 0.8672 - val_loss: 0.6458
Epoch 12/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accu

## Make prediction

In [50]:
# Be aware of change target_size to (300,300) for input_shape=(300,300,3) model
img = image.load_img('3.jpg', target_size=(150,150))
x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)   # If >0.5 is classified as human else horse
if classes[0]>0.5:
    print('is human')
else:
    print('is horse')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
is human
